# Implementation of Down Sampling by using FFT and LPF for 2D digital colorful image
Implementatotion FFT and LPF and Down Sampling without using prebuilt functions and library .Just using numpy library for math calculation .

In [1]:
import numpy as np
import cv2

def fft2D(image):
    M, N, _ = image.shape
    if M % 2 != 0 or N % 2 != 0:
        raise ValueError("Image dimensions must be even for 2D FFT.")

    # Perform 2D FFT on each color channel
    fft_result = np.zeros_like(image, dtype=np.complex128)
    for c in range(3):
        fft_result[:, :, c] = fft2D_single_channel(image[:, :, c])

    return fft_result


def fft2D_single_channel(channel):
    M, N = channel.shape

    # Perform 1D FFT on rows
    fft_rows = np.zeros_like(channel, dtype=np.complex128)
    for i in range(M):
        fft_rows[i] = fft(channel[i])

    # Perform 1D FFT on columns
    fft_result = np.zeros_like(channel, dtype=np.complex128)
    for j in range(N):
        fft_result[:, j] = fft(fft_rows[:, j])

    return fft_result


def ifft2D(image_fft):
    M, N, _ = image_fft.shape
    if M % 2 != 0 or N % 2 != 0:
        raise ValueError("Image dimensions must be even for 2D IFFT.")

    # Perform 2D IFFT on each color channel
    ifft_result = np.zeros_like(image_fft, dtype=np.complex128)
    for c in range(3):
        ifft_result[:, :, c] = ifft2D_single_channel(image_fft[:, :, c])

    return ifft_result


def ifft2D_single_channel(channel_fft):
    M, N = channel_fft.shape

    # Perform 1D IFFT on rows
    ifft_rows = np.zeros_like(channel_fft, dtype=np.complex128)
    for i in range(M):
        ifft_rows[i] = ifft(channel_fft[i])

    # Perform 1D IFFT on columns
    ifft_result = np.zeros_like(channel_fft, dtype=np.complex128)
    for j in range(N):
        ifft_result[:, j] = ifft(ifft_rows[:, j])

    return ifft_result


def fft(x):
    N = len(x)
    if N <= 1:
        return x

    even = fft(x[0::2])
    odd = fft(x[1::2])
    factor = np.exp(-2j * np.pi * np.arange(N) / N)
    return np.concatenate([even + factor[:N//2] * odd, even + factor[N//2:] * odd])


def ifft(x):
    N = len(x)
    if N <= 1:
        return x

    even = ifft(x[0::2])
    odd = ifft(x[1::2])
    factor = np.exp(2j * np.pi * np.arange(N) / N)
    return np.concatenate([even + factor[:N//2] * odd, even + factor[N//2:] * odd])


def low_pass_filter(image_fft, cutoff):
    M, N, _ = image_fft.shape
    center_i = M // 2
    center_j = N // 2

    # Apply low-pass filter
    for i in range(M):
        for j in range(N):
            if abs(i - center_i) > cutoff or abs(j - center_j) > cutoff:
                image_fft[i, j] = 0

    return image_fft


def downsample(image, scale):
    return image[::scale, ::scale, :]


## Example Usage of Down Sampling 
This example gets 2D digital colorful image input and then transform it to frequency after that use LPF (Low Pass Filter) with custom cutoff . Then transform the filtered image from frequency to time by using FFT inverse and reconstructe the filtered image .
After that down sample the reconstructed filtered image by scale factor of 2 . Before display the images should scale the pixel values of down sampled image to the range of 0-255 . 
In the last part display the original and down sampled image .

In [2]:
# Example usage
image = cv2.imread("input.jpg")  # Read input image

# Perform 2D FFT
fft_result = fft2D(image)

# Apply low-pass filter
cutoff_freq = 600
filtered_fft = low_pass_filter(fft_result, cutoff_freq)

# Perform inverse 2D IFFT
filtered_image = ifft2D(filtered_fft)
filtered_image = np.real(filtered_image)  # Take only the real part

# Downsample the filtered image
scale_factor = 2
downsampled_image = downsample(filtered_image, scale_factor)

# Scale the pixel values to the range of 0-255
downsampled_image = (downsampled_image - np.min(downsampled_image)) * (255 / np.max(downsampled_image))
downsampled_image = downsampled_image.astype(np.uint8)

#Save the input and output images
#cv2.imwrite("input_image.jpg", image)
#cv2.imwrite("output_image.jpg", downsampled_image)

# Display the images
cv2.imshow("Original Image", image)
cv2.imshow("Filtered and Downsampled Image", downsampled_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
